## 安裝 tensorflow_privacy

In [27]:
pip install tensorflow_privacy

In [0]:
import numpy as np
import tensorflow as tf

In [0]:
from privacy.analysis.rdp_accountant import compute_rdp
from privacy.analysis.rdp_accountant import get_privacy_spent
from privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer

## 下載Minst資料


In [0]:
def load_mnist():
  """Loads MNIST and preprocesses to combine training and validation data."""
  train, test = tf.keras.datasets.mnist.load_data()
  train_data, train_labels = train
  test_data, test_labels = test

  train_data = np.array(train_data, dtype=np.float32) / 255
  test_data = np.array(test_data, dtype=np.float32) / 255

  train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
  test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

  train_labels = np.array(train_labels, dtype=np.int32)
  test_labels = np.array(test_labels, dtype=np.int32)

  train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
  test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

  assert train_data.min() == 0.
  assert train_data.max() == 1.
  assert test_data.min() == 0.
  assert test_data.max() == 1.

  return train_data, train_labels, test_data, test_labels

## 不加Noise(使用原資料) -> CNN -> 檢查最後的Acc

In [37]:
def main(): 
  tf.logging.set_verbosity(tf.logging.INFO)

  # Load training and test data.
  train_data, train_labels, test_data, test_labels = load_mnist()

  # Define a sequential Keras model
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                             strides=2,
                             padding='same',
                             activation='relu',
                             input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4,
                             strides=2,
                             padding='valid',
                             activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10)
  ])
  
  
  #from distutils.version import LooseVersion
  #if LooseVersion(tf.__version__) < LooseVersion('2.0.0'):
   # GradientDescentOptimizer = tf.train.GradientDescentOptimizer
  #else:
  GradientDescentOptimizer = tf.keras.optimizers.SGD
  optimizer =  GradientDescentOptimizer(learning_rate=0.15)
    
    
  # Compute vector of per-example loss rather than its mean over a minibatch.
  loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
  
  
  # Compile model with Keras
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  
  # Train model with Keras
  model.fit(train_data, train_labels,
            epochs=5,
            validation_data=(test_data, test_labels),
            batch_size=400)

  
if __name__ == '__main__':
  main()

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 1s 18us/sample - loss: 0.9645 - acc: 0.6906 - val_loss: 0.1566 - val_acc: 0.9536
Epoch 2/5
60000/60000 [==============================] - 1s 12us/sample - loss: 0.1498 - acc: 0.9532 - val_loss: 0.0973 - val_acc: 0.9696
Epoch 3/5
60000/60000 [==============================] - 1s 12us/sample - loss: 0.1177 - acc: 0.9636 - val_loss: 0.0991 - val_acc: 0.9690
Epoch 4/5
60000/60000 [==============================] - 1s 12us/sample - loss: 0.0768 - acc: 0.9761 - val_loss: 0.0557 - val_acc: 0.9824
Epoch 5/5
60000/60000 [==============================] - 1s 12us/sample - loss: 0.0620 - acc: 0.9808 - val_loss: 0.0592 - val_acc: 0.9824


### 不加Noise後的訓練結果
**val_acc = 0.9824** <br />
**訓練時間為 = 5s**

## 加入Noise -> CNN -> 檢查最後的Acc

In [12]:
def noise_main():
  tf.logging.set_verbosity(tf.logging.INFO)

  # Load training and test data.
  train_data, train_labels, test_data, test_labels = load_mnist()

  # Define a sequential Keras model
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                             strides=2,
                             padding='same',
                             activation='relu',
                             input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4,
                             strides=2,
                             padding='valid',
                             activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10)
  ])
  
  optimizer = DPGradientDescentGaussianOptimizer(
      l2_norm_clip=1.0,
      noise_multiplier=1.1,
      num_microbatches=400,
      learning_rate=0.15)
    
     
  # Compute vector of per-example loss rather than its mean over a minibatch.
  loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True, reduction=tf.losses.Reduction.NONE)
  
  # Compile model with Keras
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  # Train model with Keras
  model.fit(train_data, train_labels,
            epochs=5,
            validation_data=(test_data, test_labels),
            batch_size=400)


if __name__ == '__main__':
  noise_main()

11493376/11490434 [==============================] - 0s 0us/step
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 60000 samples, validate on 10000 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/5
60000/60000 [==============================] - 74s 1ms/sample - loss: 1.8929 - acc: 0.4248 - val_loss: 1.2620 - val_acc: 0.6390
Epoch 2/5
60000/60000 [==============================] - 71s 1ms/sample - loss: 0.9685 - acc: 0.6933 - val_loss: 0.7297 - val_acc: 0.7667
Epoch 3/5
60000/60000 [==============================] - 71s 1ms/sample - loss: 0.6523 - acc: 0.8009 - val_loss: 0.5515 - val_acc: 0.8370
Epoch 4/5
60000/60000 [==============================] - 71s 1ms/sample - loss: 0.5418 - acc: 0.8447 - val_loss: 0.4756 - val_acc: 0.8694
Epoch 5/5
60000/60000 [==============================] - 71s 1ms/sample - loss: 0.4988 - acc: 0.8663 - val_loss: 0.4534 

### 加入Noise後的訓練結果
**val_acc = 0.8826** <br />
**訓練時間為 = 8mins**